## Sistemas Lineares Singulares ou Não-quadrados

Considere a matriz abaixo.

In [3]:
A = [1.0 2.0 0.0 2.0 5.0; 0.0 0.0 1.0 -4.0 2.0]

2x5 Array{Float64,2}:
 1.0  2.0  0.0   2.0  5.0
 0.0  0.0  1.0  -4.0  2.0

$A$ claramente tem um problema. Ela só tem duas linhas, mas tem cinco colunas.
Então podemos esperar que $Ax = b$ não tenha solução, ou que tenha infinitas.
Mas como verificar isso?

In [4]:
A = [1.0 0.0; 0.0 1.0; 1.0 1.0]

3x2 Array{Float64,2}:
 1.0  0.0
 0.0  1.0
 1.0  1.0

Esta $A$ tem um problema parecido. Suas linhas são em maior número que a coluna. O que esperar do sistema $Ax = b$?

## Imagem e Núcleo

Em Álgebra Linear, já foi visto imagem e núcleo de transformações lineares.
Também vimos que $A$ define a transformação $T:\mathbb{R}^n\longrightarrow\mathbb{R}^m$ dada
por $T(x) = Ax$. Para matrizes, estendendo o conceito, temos

$$ \mbox{Im}(A) = \{Ax \mid\ \forall x \in \mathbb{R}^n\}
= \{y \in \mathbb{R}^m \mid\ \exists x \in \mathbb{R}^m \mid\ y = Ax\}
\subset \mathbb{R}^m $$
$$ \mbox{Nu}(A) = \{x \mid\ Ax = 0\} \subset \mathbb{R}^n. $$

O sistema linear $Ax = b$ tem solução se $b \in \mbox{Im}(A)$. E caso tenha,
todas soluções são da forma $x + d$, onde $Ad = 0$, isto é, $x + \mbox{Nu}(A)$,
onde $x$ é uma solução.

No primeiro caso, podemos trocas as colunas de $A$ obtendo $[I\ B]$, onde $I$ é a identidade e $B$ é uma matriz qualquer. Trocamos as linhas de $x$ respectivamente, obtendo $[x_I; x_B]$. Uma solução do sistema $Ax = b$ seria
$x_I = b$ e $x_B = 0$.

De maneira geral, $I$ poderia ser $U$ e fariamos $x_I = U^{-1}b$.

Uma estratégia para obter isso é realmente fazer as permutações de coluna na fatoração LU.
Isso é chamado de pivoteamento completo.
Apesar de funcionar, é pouco praticado.

Def.: (Posto) O posto-linha da matriz $A$ é o número de linhas linearmente independente de $A$.
O posto-coluna é análogo.

Teo.: O posto-linha é igual ao posto-coluna e por isso dizemos apenas posto.

Def.: Dizemos que uma matriz $A: m\times n$ tem posto-linha completo de o posto de $A$ é $m$.
Análogo para posto-coluna completo.

Def.: Dizemos que $A$ tem posto completo se o posto de $A$ é $\min\{m,n\}$.

Teo.: Se $A$ tem posto-linha completo, então $Ax = b$ tem uma solução se $n = m$ ou infinitas
se $m < n$. (Note que $m > n$ é impossível.)

Teo.: Se $A$ tem posto-coluna completo, então $\mbox{Nu}(A) = \{0\}$. Daí, se existir solução,
ela é única. Note que pode não existir solução.

Teo.: Se $A$, $m\times n$ com $m \leq n$, então existem matrizes $L$, $U$ e $P$ tais que
$PA = LU$, com $P$ matriz de permutação de linhas, $L$ triangular inferior $m\times m$ com diagonal de 1s e
$U$ é tipo escada superior $m\times n$ com o mesmo posto que $A$.

In [10]:
A = [2.0 3.0 1.0 1.0 1.0; -4.0 -6.0 -3.0 3.0 2.0]

2x5 Array{Float64,2}:
  2.0   3.0   1.0  1.0  1.0
 -4.0  -6.0  -3.0  3.0  2.0

In [11]:
m21 = A[2,1]/A[1,1]
U = copy(A)
U[2,1] = 0.0
U[2,2:5] -= m21*U[1,2:5]
U

2x5 Array{Float64,2}:
 2.0  3.0   1.0  1.0  1.0
 0.0  0.0  -1.0  5.0  4.0

In [15]:
A = [2.0 3.0 1.0 1.0 1.0; 0.0 0.0 -1.0 5.0 4.0; 0.0 0.0 2.0 -10.0 -8.0]

3x5 Array{Float64,2}:
 2.0  3.0   1.0    1.0   1.0
 0.0  0.0  -1.0    5.0   4.0
 0.0  0.0   2.0  -10.0  -8.0

In [16]:
m33 = A[3,3]/A[2,3]
U = copy(A)
U[3,3] = 0.0
U[3,4:end] -= m33*U[2,4:end]
U

3x5 Array{Float64,2}:
 2.0  3.0   1.0  1.0  1.0
 0.0  0.0  -1.0  5.0  4.0
 0.0  0.0   0.0  0.0  0.0

Teo.: Se $A$ é $m \times n$ com $m > n$, então existe $L$, $m\times n$ com diagonal de 1
e $U$, $n \times n$ com mesmo posto que $A$, $P$ permutação de linhas tais que 
$PA = LU$.

In [30]:
A = rand(5,3)

5x3 Array{Float64,2}:
 0.439409  0.151356   0.41955 
 0.812813  0.997602   0.996961
 0.453936  0.102023   0.652977
 0.402581  0.220917   0.529434
 0.402738  0.0837114  0.293798

In [38]:
function dec_lu_pivot(A; tol=1e-12)
    m,n = size(A)
    U = copy(A)
    L = zeros(m,n)
    P = collect(1:m)
    for j = 1:n
        if j > m
            break
        end
        i_pivo, pivo = j, abs(U[j,j])
        for i = j+1:m
            if abs(U[i,j]) > pivo
                pivo = abs(U[i,j])
                i_pivo = i
            end
        end
        if pivo < tol
            error("pivo ≈ 0.0")
        end
        if i_pivo != j
            U[[i_pivo; j],:] = U[[j; i_pivo],:]
            L[[i_pivo; j],:] = L[[j; i_pivo],:]
            P[[i_pivo; j]] = P[[j; i_pivo]]
        end
        L[j,j] = 1.0
        for i = j+1:m
            mij = U[i,j]/U[j,j]
            L[i,j] = mij
            U[i,j] = 0.0
            for k = j+1:n
                U[i,k] -= mij*U[j,k]
            end
        end
    end
    return L, U, P
end

dec_lu_pivot (generic function with 1 method)

In [40]:
# Testes
A = rand(3,5)
L, U, P = dec_lu_pivot(A)

(
3x5 Array{Float64,2}:
 1.0        0.0       0.0  0.0  0.0
 0.0300684  1.0       0.0  0.0  0.0
 0.711383   0.633366  1.0  0.0  0.0,

3x5 Array{Float64,2}:
 0.645079  0.585493  0.174164   0.784098   0.542631 
 0.0       0.858781  0.505681   0.413835   0.0453217
 0.0       0.0       0.457704  -0.362137  -0.0562869,

[2,3,1])

In [42]:
A = rand(3,5)
A[3,:] = A[1,:] + A[2,:]
L, U, P = dec_lu_pivot(A)

LoadError: LoadError: pivo ≈ 0.0
while loading In[42], in expression starting on line 3

In [43]:
A = rand(5,3)
L, U, P = dec_lu_pivot(A)

(
5x3 Array{Float64,2}:
 1.0         0.0       0.0     
 0.0312529   1.0       0.0     
 0.906889   -0.204412  1.0     
 0.96666    -0.023116  0.976029
 0.490456    0.147277  0.198952,

5x3 Array{Float64,2}:
 0.992183  0.244603  0.536551
 0.0       0.925348  0.246548
 0.0       0.0       0.245945
 0.0       0.0       0.0     
 0.0       0.0       0.0     ,

[1,2,4,3,5])

In [45]:
A = rand(5,3)
A[:,3] = A[:,1] + A[:,2]
L, U, P = dec_lu_pivot(A)

LoadError: LoadError: pivo ≈ 0.0
while loading In[45], in expression starting on line 3

In [175]:
function dec_lu_pivot(A; tol=1e-12)
    m,n = size(A)
    U = copy(A)
    if m <= n
        L = zeros(m,m)
    else
        L = zeros(m,n)
    end
    P = collect(1:m)
    d = 1 # Linha onde escada deve ser não-nulo. Se não tiver problemas, d=j, senão d<j.
    for j = 1:n
        i_pivo, pivo = d, abs(U[d,j])
        for i = d+1:m
            if abs(U[i,j]) > pivo
                pivo = abs(U[i,j])
                i_pivo = i
            end
        end
        if pivo < tol
            # Ninguém nessa coluna é não-nulo o suficiente.
            # Vamos dizer que são todos zero, então.
            for i = d:m
                U[i,j] = 0.0
            end
            continue
        end
        if i_pivo != d
            U[[i_pivo; d],:] = U[[d; i_pivo],:]
            L[[i_pivo; d],:] = L[[d; i_pivo],:]
            P[[i_pivo; d]] = P[[d; i_pivo]]
        end
        for i = d+1:m
            mid = U[i,j]/U[d,j]
            L[i,d] = mid
            U[i,j] = 0.0
            for k = j+1:n
                U[i,k] -= mid*U[d,k]
            end
        end
        d += 1
        if d > m
            break
        end
    end
    for i = 1:min(m,n)
        L[i,i] = 1.0
    end
    if m > n
        U = U[1:n,:]
    end
    return L, U, P
end

dec_lu_pivot (generic function with 1 method)

In [176]:
# Testes
A = rand(3,5)
L, U, P = dec_lu_pivot(A)

(
3x3 Array{Float64,2}:
 1.0       0.0       0.0
 0.239692  1.0       0.0
 0.550198  0.640936  1.0,

3x5 Array{Float64,2}:
 0.657599  0.51093  0.712492     0.757844   0.0236808
 0.0       0.13412  0.00860452   0.732448   0.450144 
 0.0       0.0      0.385824    -0.0770405  0.447778 ,

[3,2,1])

In [177]:
L*U - A[P,:]

3x5 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0

In [178]:
A = rand(3,5)
A[3,:] = A[1,:] + A[2,:]
L, U, P = dec_lu_pivot(A)

(
3x3 Array{Float64,2}:
 1.0         0.0  0.0
 0.0518788   1.0  0.0
 0.948121   -1.0  1.0,

3x5 Array{Float64,2}:
 0.766777  0.177805  0.59293    0.830237   0.669112  
 0.0       0.116375  0.0866571  0.403018  -0.00366955
 0.0       0.0       0.0        0.0        0.0       ,

[3,2,1])

In [179]:
L*U - A[P,:]

3x5 Array{Float64,2}:
 0.0  0.0  0.0          0.0   0.0        
 0.0  0.0  0.0          0.0   0.0        
 0.0  0.0  5.55112e-17  0.0  -1.11022e-16

In [180]:
A = rand(5,7)
A[3,1:6] = 0.0
A[4,1:4] = 0.0
A[5,1:3] = 0.0
A[3,:] += A[1,:]
A[4,:] += A[1,:] + 2*A[2,:]
A[5,:] += -A[1,:] + 3*A[2,:]
A

5x7 Array{Float64,2}:
 0.171858  0.843472  0.511865  0.0590484  0.334704  0.259002  0.956345
 0.626128  0.510848  0.869225  0.569453   0.226876  0.949505  0.746139
 0.171858  0.843472  0.511865  0.0590484  0.334704  0.259002  1.07553 
 1.42411   1.86517   2.25032   1.19795    1.42983   2.26075   2.73661 
 1.70653   0.689071  2.09581   2.1145     1.02266   3.0566    2.20711 

In [181]:
L, U, P = dec_lu_pivot(A)

(
5x5 Array{Float64,2}:
 1.0       0.0   0.0   0.0  0.0
 0.834511  1.0   0.0   0.0  0.0
 0.100706  0.6   1.0   0.0  0.0
 0.366902  0.2  -0.5   1.0  0.0
 0.100706  0.6   1.0  -0.0  1.0,

5x7 Array{Float64,2}:
 1.70653  0.689071  2.09581   2.1145     1.02266    3.0566      2.20711  
 0.0      1.29013   0.50134  -0.566622   0.576404  -0.290015    0.894745 
 0.0      0.0       0.0       0.186077  -0.114128   0.125192    0.316408 
 0.0      0.0       0.0       0.0       -0.320685  -0.0513676  -0.0844008
 0.0      0.0       0.0       0.0        0.0        0.0        -0.11918  ,

[5,4,3,2,1])

In [182]:
L*U - A[P,:]

5x7 Array{Float64,2}:
 0.0  0.0          0.0          0.0          0.0  0.0  0.0        
 0.0  0.0          0.0          0.0          0.0  0.0  0.0        
 0.0  1.11022e-16  0.0          2.77556e-17  0.0  0.0  0.0        
 0.0  0.0          2.22045e-16  0.0          0.0  0.0  1.11022e-16
 0.0  1.11022e-16  0.0          2.77556e-17  0.0  0.0  0.0        

In [183]:
A = rand(5,7)
A[1,1:6] = 0.0
A[2,1:3] = 0.0
A[2,5:end] = 0.0
A[3,1:5] = 0.0
A[1,:] += A[4,:]
A[2,:] += A[5,:] + 2*A[4,:]
A[3,:] += -A[5,:] + 3*A[4,:]
A

5x7 Array{Float64,2}:
 0.739799   0.0663324  0.523188  0.428924  0.564921  0.797903  1.68936 
 1.98711    0.428244   1.67945   1.47115   1.26209   2.24101   1.90183 
 1.71188   -0.0965818  0.936489  0.681168  1.56251   1.79893   2.80253 
 0.739799   0.0663324  0.523188  0.428924  0.564921  0.797903  0.89869 
 0.507516   0.295579   0.633074  0.605604  0.132252  0.645205  0.104451

In [184]:
L, U, P = dec_lu_pivot(A)

(
5x5 Array{Float64,2}:
 1.0        0.0  0.0       0.0  0.0
 0.861491   1.0  0.0       0.0  0.0
 0.255404  -0.4  1.0       0.0  0.0
 0.372298   0.2  0.333333  1.0  0.0
 0.372298   0.2  0.333333  1.0  1.0,

5x7 Array{Float64,2}:
 1.98711   0.428244   1.67945   …  1.26209       2.24101     1.90183  
 0.0      -0.46551   -0.510341     0.475228     -0.131678    1.16413  
 0.0       0.0        0.0          8.32667e-17   0.0201711   0.0843667
 0.0       0.0        0.0          0.0          -0.0168092  -0.0703056
 0.0       0.0        0.0          0.0           0.0         0.790675 ,

[2,3,5,4,1])

In [185]:
L*U - A[P,:]

5x7 Array{Float64,2}:
 0.0  0.0   0.0          0.0  0.0  0.0   0.0        
 0.0  0.0   0.0          0.0  0.0  0.0   0.0        
 0.0  0.0  -1.11022e-16  0.0  0.0  0.0  -2.77556e-17
 0.0  0.0   0.0          0.0  0.0  0.0   0.0        
 0.0  0.0   0.0          0.0  0.0  0.0   0.0        

In [186]:
A = rand(5,3)
L, U, P = dec_lu_pivot(A)

(
5x3 Array{Float64,2}:
 1.0         0.0         0.0     
 0.0903308   1.0         0.0     
 0.883046    0.80568     1.0     
 0.768573   -0.00209503  0.207168
 0.383962    0.699203    0.414936,

3x3 Array{Float64,2}:
 0.913964  0.209518  0.0312528
 0.0       0.888529  0.314393 
 0.0       0.0       0.431329 ,

[5,4,2,3,1])

In [188]:
L*U - A[P,:]

5x3 Array{Float64,2}:
 0.0   0.0          0.0        
 0.0   0.0          0.0        
 0.0  -1.11022e-16  0.0        
 0.0   0.0          1.38778e-17
 0.0   0.0          5.55112e-17

In [190]:
A = rand(5,3)
A[:,3] = A[:,1] + A[:,2]
A

5x3 Array{Float64,2}:
 0.249174  0.938664   1.18784 
 0.946152  0.0469359  0.993087
 0.516752  0.143781   0.660533
 0.733684  0.548442   1.28213 
 0.351406  0.0114379  0.362844

In [191]:
L, U, P = dec_lu_pivot(A)

(
5x3 Array{Float64,2}:
 1.0        0.0         0.0
 0.263355   1.0         0.0
 0.546162   0.127547    1.0
 0.77544    0.552785    0.0
 0.371405  -0.00647127  0.0,

3x3 Array{Float64,2}:
 0.946152  0.0469359  0.993087
 0.0       0.926303   0.926303
 0.0       0.0        0.0     ,

[2,1,3,4,5])

In [193]:
L*U - A[P,:]

5x3 Array{Float64,2}:
 0.0  0.0          0.0        
 0.0  0.0          0.0        
 0.0  0.0          1.11022e-16
 0.0  1.11022e-16  2.22045e-16
 0.0  0.0          0.0        

In [196]:
A = rand(9,5)
A[1,:] = 0
A[:,1] = 0
A[:,3] = -2*A[:,4]
A[5,:] = A[4,:] + A[6,:]
A[7,3:5] = 0
A[7,:] += A[8,:]
A

9x5 Array{Float64,2}:
 0.0  0.0         -0.0       0.0        0.0     
 0.0  0.283638    -0.777501  0.388751   0.430303
 0.0  0.967686    -0.129235  0.0646177  0.562396
 0.0  0.921316    -1.75386   0.876928   0.16809 
 0.0  1.867       -2.91008   1.45504    0.445909
 0.0  0.945686    -1.15622   0.578112   0.277819
 0.0  0.66428     -1.52715   0.763577   0.309533
 0.0  0.00518196  -1.52715   0.763577   0.309533
 0.0  0.88895     -1.99609   0.998047   0.976709

In [197]:
L, U, P = dec_lu_pivot(A)

(
9x5 Array{Float64,2}:
 1.0          0.0        0.0        0.0  0.0
 0.00277555   1.0        0.0        0.0  0.0
 0.476138     0.401886   1.0        0.0  0.0
 0.493474     0.209211  -0.181817   1.0  0.0
 0.0         -0.0        0.0        0.0  1.0
 0.506526    -0.209211   0.181817   0.0  0.0
 0.355801     0.323714   0.0797492  0.0  0.0
 0.151921     0.220791   0.459787   0.0  0.0
 0.51831     -0.907846   0.954202   0.0  0.0,

5x5 Array{Float64,2}:
 0.0  1.867  -2.91008  1.45504   0.445909
 0.0  0.0    -1.51908  0.759538  0.308296
 0.0  0.0     0.0      0.0       0.640495
 0.0  0.0     0.0      0.0       0.0     
 0.0  0.0     0.0      0.0       0.0     ,

[5,8,9,4,1,6,7,2,3])

In [198]:
L*U - A[P,:]

9x5 Array{Float64,2}:
 0.0  0.0           0.0          0.0          0.0
 0.0  0.0           0.0          0.0          0.0
 0.0  0.0           0.0          0.0          0.0
 0.0  0.0           0.0          0.0          0.0
 0.0  0.0           0.0          0.0          0.0
 0.0  0.0           0.0          0.0          0.0
 0.0  0.0           0.0          0.0          0.0
 0.0  0.0           0.0          0.0          0.0
 0.0  1.11022e-16  -8.32667e-17  4.16334e-17  0.0

É possível reduzir ainda mais.

In [205]:
L = tril(rand(8,3),-1) + eye(8,3)

8x3 Array{Float64,2}:
 1.0        0.0        0.0     
 0.333354   1.0        0.0     
 0.0801818  0.0864513  1.0     
 0.587797   0.0114004  0.737879
 0.901065   0.127944   0.141796
 0.922428   0.725448   0.534785
 0.334726   0.13626    0.620181
 0.14462    0.590013   0.335917

In [208]:
U = triu(rand(3,8))

3x8 Array{Float64,2}:
 0.624965  0.0462707  0.936622  0.700894  …  0.425097   0.937541  0.450068
 0.0       0.455775   0.337873  0.235228     0.774182   0.269371  0.13573 
 0.0       0.0        0.68337   0.515467     0.0773823  0.53236   0.701346

In [209]:
A = L*U

8x8 Array{Float64,2}:
 0.624965   0.0462707  0.936622  0.700894  …  0.425097  0.937541  0.450068
 0.208335   0.471199   0.6501    0.468874     0.91589   0.581904  0.285762
 0.0501108  0.0431124  0.78768   0.592002     0.178396  0.630821  0.749167
 0.367353   0.0323938  1.05864   0.795018     0.315796  0.946972  0.783604
 0.563134   0.100006   0.984085  0.734739     0.493064  0.954737  0.522355
 0.576486   0.373323   1.47453   1.09283   …  0.995134  1.34493   0.888689
 0.209192   0.0775919  0.783364  0.586343     0.295772  0.680684  0.604106
 0.0903828  0.275605   0.56436   0.413306     0.544249  0.473349  0.380766

In [210]:
L, U, P = dec_lu_pivot(A)

(
8x8 Array{Float64,2}:
 1.0        0.0        0.0       0.0  0.0  0.0  0.0  0.0
 0.333354   1.0        0.0       0.0  0.0  0.0  0.0  0.0
 0.0801818  0.0864513  1.0       0.0  0.0  0.0  0.0  0.0
 0.587797   0.0114004  0.737879  1.0  0.0  0.0  0.0  0.0
 0.901065   0.127944   0.141796  0.0  1.0  0.0  0.0  0.0
 0.922428   0.725448   0.534785  0.0  0.0  1.0  0.0  0.0
 0.334726   0.13626    0.620181  0.0  0.0  0.0  1.0  0.0
 0.14462    0.590013   0.335917  0.0  0.0  0.0  0.0  1.0,

8x8 Array{Float64,2}:
 0.624965  0.0462707  0.936622  0.700894  …  0.425097   0.937541  0.450068
 0.0       0.455775   0.337873  0.235228     0.774182   0.269371  0.13573 
 0.0       0.0        0.68337   0.515467     0.0773823  0.53236   0.701346
 0.0       0.0        0.0       0.0          0.0        0.0       0.0     
 0.0       0.0        0.0       0.0          0.0        0.0       0.0     
 0.0       0.0        0.0       0.0       …  0.0        0.0       0.0     
 0.0       0.0        0.0       0.0          0